# Implementing Sauerland's algorithm using the pointed-set monad

In [1]:
import Prelude hiding (lookup)

## Syntax

In this system, we won't be treating exhaustification as a unary logical operator, but ultimately as a the reflex of a special interpretation rule. As such, the syntax for propositional logic is rather standard, the only difference being the presence of the binary connectives `L` and `R` (following Sauerland 2004).

In [2]:
newtype Var = Var Char
    deriving (Eq, Ord)
    
data Expr = Variable Var
          | Negation Expr
          | Conjunction Expr Expr
          | Disjunction Expr Expr
          | L Expr Expr
          | R Expr Expr
          deriving Eq
          
mkVar :: Char -> Expr
mkVar = Variable . Var

Here, we define some functions in order to print formulas of propositional logic in a readable fashion.

In [3]:
instance Show Var where
    show (Var v) = [v]

showBinaryConnective :: (Expr -> String) -> String -> Expr -> Expr -> String
showBinaryConnective show_ symbol exp1 exp2 =
  '(' : show_ exp1 ++ " " ++ symbol ++ " " ++ show_ exp2 ++ ")"

showBC :: String -> Expr -> Expr -> String
showBC = showBinaryConnective show
          
instance Show Expr where
  show (Variable      name)      = show name
  show (Negation      expr)      = '~' : show expr
  show (Conjunction   exp1 exp2) = showBC "&&" exp1 exp2
  show (Disjunction   exp1 exp2) = showBC "||" exp1 exp2
  show (L   exp1 exp2) = showBC "L" exp1 exp2
  show (R   exp1 exp2) = showBC "R" exp1 exp2

## Computing structural alternatives

Here, we use Chung-chieh Shan's *pointed-set monad* in order to compute structural alternatives in a compositional fashion (definitions taken from [this blog post](http://conway.rutgers.edu/~ccshan/wiki/blog/posts/Pointed_set)). The pointed-set monad is used to model a monadic computation that keeps track of both $a$s and indeterminate $a$s (in this instance, the set of alternatives).

In [4]:
import Control.Monad (liftM,replicateM,ap)

data Pointed a = Pointed a [a]
  deriving (Eq, Ord, Show, Read)

toList (Pointed x xs) = x:xs

instance Functor Pointed where
  fmap = liftM

instance Applicative Pointed where
  pure = return
  (<*>) = ap
  
instance Monad Pointed where
  return x = Pointed x []
  Pointed x xs >>= k = Pointed y (ys ++ (xs >>= toList.k))
    where Pointed y ys = k x

In [42]:
binaryConnectives :: [Expr -> Expr -> Expr]
binaryConnectives = [Conjunction, Disjunction, L, R]

alts :: (Expr -> Expr -> Expr) -> Pointed (Expr -> Expr -> Expr)
alts op = Pointed op binaryConnectives

In [47]:
getAlts :: Pointed Expr -> [Expr]
getAlts (Pointed p alts) = alts

disjAlts = getAlts $ alts Disjunction <*> (alts Disjunction <*> pure (mkVar 'a') <*> pure (mkVar 'b')) <*> pure (mkVar 'c')

disjAlts

[((a && b) || c),((a || b) || c),((a L b) || c),((a R b) || c),((a || b) && c),((a && b) && c),((a || b) && c),((a L b) && c),((a R b) && c),((a || b) || c),((a && b) || c),((a || b) || c),((a L b) || c),((a R b) || c),((a || b) L c),((a && b) L c),((a || b) L c),((a L b) L c),((a R b) L c),((a || b) R c),((a && b) R c),((a || b) R c),((a L b) R c),((a R b) R c)]

Below, we compositionally compute the set of alternatives for the formula `a || b || c`.

In [6]:
alts Disjunction <*> (alts Disjunction <*> pure (mkVar 'a') <*> pure (mkVar 'b')) <*> pure (mkVar 'c')

Pointed ((a || b) || c) [((a && b) || c),((a || b) || c),((a L b) || c),((a R b) || c),((a || b) && c),((a && b) && c),((a || b) && c),((a L b) && c),((a R b) && c),((a || b) || c),((a && b) || c),((a || b) || c),((a L b) || c),((a R b) || c),((a || b) L c),((a && b) L c),((a || b) L c),((a L b) L c),((a R b) L c),((a || b) R c),((a && b) R c),((a || b) R c),((a L b) R c),((a R b) R c)]

## Semantics

Below you can find the basic semantics for propositional logic. The only unusual feature is that `a L b` simply returns the value of `a` relative to the assignment, and `a R b` returns the value of `b` relative to the assignment.

In [7]:
import Data.Map (Map,fromList,lookup)
import Data.List (group,sort)
import Data.Maybe (fromMaybe)

type Mapping = Map Var Bool

variables :: Expr -> [Var]
variables expr = let vars_ (Variable      v)     vs = v : vs
                     vars_ (Negation      e)     vs = vars_ e vs
                     vars_ (Conjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (Disjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (L   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (R   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs

                 in  map head . group . sort $ vars_ expr []
                 
assignments :: Expr -> [Mapping]
assignments expr = let vs = variables expr
                       ps = replicateM (length vs) [True, False]
                   in  map (fromList . zip vs) ps
                   
assignments' :: [Var] -> [Mapping]
assignments' vs = map (fromList . zip vs) $ replicateM (length vs) [True, False]
                   
interpret :: Expr -> Mapping -> Bool
interpret (Variable      v)         vs = fromMaybe False (lookup v vs)
interpret (Negation      expr)      vs = not $ interpret expr vs
interpret (Conjunction   exp1 exp2) vs = interpret exp1 vs && interpret exp2 vs
interpret (Disjunction   exp1 exp2) vs = interpret exp1 vs || interpret exp2 vs
interpret (L   exp1 exp2) vs = interpret exp1 vs
interpret (R   exp1 exp2) vs = interpret exp2 vs

values :: Expr -> [Bool]
values expr = map (interpret expr) (assignments expr)

-- | Determines whether an expression is contradictory.
isContradiction :: Expr -> Bool
isContradiction = not . or . values

### Implementing an exhaustification operator based on minimal worlds

In [8]:
proposition :: Expr -> [Mapping] -> [Mapping]
proposition expr model = [w | w <- model, interpret expr w]

In [9]:
entails :: Expr -> Expr -> Bool
expr `entails` expr' = all (`elem` proposition expr' (assignments' (variables expr ++ variables expr'))) (proposition expr (assignments' (variables expr ++ variables expr')))

In [84]:
isPropersubset :: Eq a => [a] -> [a] -> Bool
isPropersubset x y = all (`elem` y) x && x /= y

strictPreorder :: [Expr] -> Mapping -> Mapping -> Ordering
strictPreorder alts u v
    | uAlts `isPropersubset` vAlts = LT
    | vAlts `isPropersubset` uAlts = GT
    | otherwise = EQ
        where uAlts = [p | p <- alts, interpret p u]
              vAlts = [q | q <- alts, interpret q v]

In [91]:
exhMw :: Pointed Expr -> Mapping -> Bool
exhMw (Pointed p alts) w = interpret p w && null [v|v <- assignments p, interpret p v && strictPreorder alts v w == LT]

valuesExhMw :: Pointed Expr -> [Bool]
valuesExhMw (Pointed p alts) = map (exhMw (Pointed p alts)) (assignments p)

disjunctionF :: Pointed (Expr -> Expr -> Expr)
disjunctionF = alts Disjunction

aAndB = (disjunctionF <*> (pure $ mkVar 'a')) <*> (pure $ mkVar 'b')


valuesExhMw aOrB

Line 10: Move brackets to avoid $
Found:
(disjunctionF <*> (pure $ mkVar 'a')) <*> (pure $ mkVar 'b')
Why not:
(disjunctionF <*> (pure $ mkVar 'a')) <*> pure (mkVar 'b')Line 10: Move brackets to avoid $
Found:
disjunctionF <*> (pure $ mkVar 'a')
Why not:
disjunctionF <*> pure (mkVar 'a')

[False,True,True,False]

### Semantics of exhaustification

Simple exhaustification of non-weaker alternatives.

In [58]:
excl :: Expr -> Expr -> Bool
excl s alt = not . isContradiction $ s `Conjunction` Negation alt

filterExcl :: Pointed Expr -> Pointed Expr
filterExcl (Pointed p alts) = Pointed p (filter (excl p) alts)

implicate :: Pointed Expr -> Mapping -> Bool
implicate (Pointed p alts) vs = foldr ((&&) . not . (`interpret` vs)) (interpret p vs) alts

interpretExh :: Pointed Expr -> Mapping -> Bool
interpretExh = implicate . filterExcl

valuesExh :: Pointed Expr -> [Bool]
valuesExh (Pointed p alts) = map (interpretExh (Pointed p alts)) (assignments p)

In [66]:
aOrB = alts Disjunction <*> pure (mkVar 'a') <*> pure (mkVar 'b')

In [67]:
valuesExh aOrB

[False,False,False,False]

### Truth-tables

In [15]:
type Printer = (Expr -> String, Bool -> String)

-- | The 'truthTable' function produces a truth table for the given expression.
truthTable :: Expr -> String
truthTable = truthTableP (show, showBool)

-- | The 'truthTableP' is a configurable version of 'truthTable' which allows a
-- printer function to be selected, so for example one can print ASCII truth
-- tables by passing 'showAscii' to 'truthTableP' instead of 'show'.
truthTableP :: Printer -> Expr -> String
truthTableP (expPrinter, boolPrinter) expr = unlines [header, separator, body]
  where
    header    = (unwords . map show) vs ++ " | " ++ expPrinter expr
    body      = init . unlines $ map (showAssignment boolPrinter expr) as
    separator = concat $ replicate sepLength "-"
    sepLength = length vs * 2 + length (expPrinter expr) + 2
    as        = assignments expr
    vs        = variables   expr
    
showAssignment :: (Bool -> String) -> Expr -> Mapping -> String
showAssignment printer expr a = showVarValues ++ " | " ++ showExprValue
  where
    showVarValues = unwords $ foldr ((:) . printer) [] a
    showExprValue = printer $ interpret expr a

-- | Prints @T@ for 'True' and @F@ for 'False'.
showBool :: Bool -> String
showBool True  = "T"
showBool False = "F"

In [16]:
IHaskell.Display.plain $ truthTable $ mkVar 'a' `Disjunction` mkVar 'b'

a b | (a || b)
--------------
T T | T
T F | T
F T | T
F F | F

In [17]:
showExh :: Pointed Expr -> String
showExh (Pointed p alts) = "Exh " ++ show p

In [18]:
showExh aOrB

"Exh ((a || b) || c)"

In [19]:
type PrinterExh = (Pointed Expr -> String, Bool -> String)

-- | The 'truthTable' function produces a truth table for the given expression.
truthTableExh :: Pointed Expr -> String
truthTableExh = truthTablePExh (showExh, showBool)

-- | The 'truthTableP' is a configurable version of 'truthTable' which allows a
-- printer function to be selected, so for example one can print ASCII truth
-- tables by passing 'showAscii' to 'truthTableP' instead of 'show'.
truthTablePExh :: PrinterExh -> Pointed Expr -> String
truthTablePExh (expPrinter, boolPrinter) (Pointed expr alts) = unlines [header, separator, body]
  where
    header    = (unwords . map show) vs ++ " | " ++ expPrinter (Pointed expr alts)
    body      = init . unlines $ map (showAssignment boolPrinter (Pointed expr alts)) as
    separator = concat $ replicate sepLength "-"
    sepLength = length vs * 2 + length (expPrinter (Pointed expr alts)) + 2
    as        = assignments expr
    vs        = variables   expr
    
showAssignment :: (Bool -> String) -> Pointed Expr -> Mapping -> String
showAssignment printer expr a = showVarValues ++ " | " ++ showExprValue
  where
    showVarValues = unwords $ foldr ((:) . printer) [] a
    showExprValue = printer $ interpretExh expr a

-- | Prints @T@ for 'True' and @F@ for 'False'.
showBool :: Bool -> String
showBool True  = "T"
showBool False = "F"

In [62]:
IHaskell.Display.plain $ truthTableExh aOrB

a b c | Exh ((a || b) || c)
---------------------------
T T T | F
T T F | F
T F T | F
T F F | F
F T T | F
F T F | F
F F T | F
F F F | F